<a href="https://colab.research.google.com/github/MayerT1/Sewanee_Colab/blob/main/ARISE/ARISE_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.5 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Writing/Grants/grants/Appalachian Regional Commission (ARC) Appalachian Regional Initiative for Stronger Economies (ARISE)

/content/drive/MyDrive/Writing/Grants/grants/Appalachian Regional Commission (ARC) Appalachian Regional Initiative for Stronger Economies (ARISE)


In [3]:
# Authenticate and initialize Earth Engine
import ee
import geemap
import geemap.chart as chart
ee.Authenticate()
ee.Initialize(project='servir-sco-assets')
Map = geemap.Map()

In [4]:
# Create interactive map
Map = geemap.Map(center=[34.5, -86.8], zoom=6)  # Center near AL/TN

# -------------------------------
# 1. Load State Boundaries (US Census TIGER)
# -------------------------------
# Load states
states = ee.FeatureCollection("TIGER/2018/States")
alabama = states.filter(ee.Filter.eq('NAME', 'Alabama'))
tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))

# Style for no fill + black boundary
outline_style = {
    'color': '000000',      # black outline
    'width': 2,             # line thickness
    'fillColor': '00000000' # transparent fill (00 = zero opacity)
}

# Apply style
alabama_outline = alabama.style(**outline_style)
tennessee_outline = tennessee.style(**outline_style)

# Add to map
Map.addLayer(alabama_outline, {}, "Alabama Boundary")
Map.addLayer(tennessee_outline, {}, "Tennessee Boundary")



# -------------------------------
# 4. Forest Cover (Use Hansen Global Forest Change)
# -------------------------------
forest = ee.Image("UMD/hansen/global_forest_change_2022_v1_10") \
            .select('treecover2000')

forest_vis = {
    'min': 0,
    'max': 100,
    'palette': ['white', 'green']
}

Map.addLayer(forest, forest_vis, "Forest Cover %")


# -------------------------------
# 7. EPA Level IV Ecoregions
# -------------------------------

eco = ee.FeatureCollection("EPA/Ecoregions/2013/L4")

# Convert polygons → image by painting with shape_area
eco_image = ee.Image().float().paint(eco, 'shape_area')

# Visualization parameters (translated from JS)
eco_vis = {
    'min': 16875853.2756,
    'max': 23201856922.5,
    'palette': ["#0a3b04", "#1a9924", "#15d812"], # deep green → bright green
    'opacity': 0.45  # lower opacity to avoid overpowering counties & GHSL
}

# Add to map
Map.addLayer(eco_image, eco_vis, "EPA Level IV Ecoregions")


# -------------------------------
# 4. Urban
# -------------------------------

# Load GHSL Built-up dataset
ghsl = ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_C").select('built_characteristics').first()

# Mask out open-space vegetation classes (1, 2, 3)
ghsl_urban = ghsl.updateMask(ghsl.gt(3))

# Visualization dictionary using your category colors
ghsl_viz = {
    "bands": ["built_characteristics"],
    "min":3,
    "max": 25,
    "palette": [
        # "#718c6c",  # 1  open spaces low veg
        # "#8ad86b",  # 2  open spaces med veg
        # "#c1ffa1",  # 3  open spaces high veg
        "#01b7ff",  # 4  open spaces water
        "#ffd501",  # 5  open spaces roads

        "#d28200",  # 11 built residential <=3m
        "#fe5900",  # 12 built residential 3-6m
        "#ff0101",  # 13 built residential 6-15m
        "#ce001b",  # 14 built residential 15-30m
        "#7a000a",  # 15 built residential >30m

        "#ff9ff4",  # 21 built non-res <=3m
        "#ff67e4",  # 22 built non-res 3-6m
        "#f701ff",  # 23 built non-res 6-15m
        "#a601ff",  # 24 built non-res 15-30m
        "#6e00fe"   # 25 built non-res >30m
    ]
}

# Add to map
Map.addLayer(ghsl_urban, ghsl_viz, "GHSL Built Form (2018)")


# -------------------------------
# 2. Counties
# -------------------------------
# counties = ee.FeatureCollection("TIGER/2018/Counties")
# Map.addLayer(counties, {'color': 'yellow'}, "Counties")


# TIGER counties collection
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Define county lists grouped by category
transitional = [
    ("Franklin", "Tennessee"), ("Marion", "Tennessee"),
    ("Coffee", "Tennessee"), ("Marshall", "Alabama"),
    ("Jackson", "Alabama")
]

at_risk = [
    ("Sequatchie", "Tennessee"), ("Warren", "Tennessee"),
    ("DeKalb", "Tennessee"), ("Van Buren", "Tennessee")
]

distressed = [
    ("Grundy", "Tennessee"), ("Bledsoe", "Tennessee")
]

competitive = [
    ("Madison", "Alabama")
]

# Function to filter counties by lists
def get_counties(list_pairs):
    fc = ee.FeatureCollection([])
    for name, state in list_pairs:
        fc = fc.merge(counties.filter(
            ee.Filter.And(
                ee.Filter.eq('NAME', name),
                ee.Filter.eq('STATEFP',
                    ee.String(
                        ee.Feature(
                            ee.FeatureCollection("TIGER/2018/States")
                                .filter(ee.Filter.eq("NAME", state))
                                .first()
                        ).get("STATEFP")
                    )
                )
            )
        ))
    return fc

# Create groups
distressed_fc = get_counties(distressed)
at_risk_fc = get_counties(at_risk)
transitional_fc = get_counties(transitional)
competitive_fc = get_counties(competitive)

# Visualize (partial transparency)
Map.addLayer(distressed_fc, {'color': '#ca727f', 'opacity': 0.6}, "Distressed")
Map.addLayer(at_risk_fc, {'color': '#eac69e', 'opacity': 0.6}, "At-Risk")
Map.addLayer(transitional_fc, {'color': '#e9ebd4', 'opacity': 0.6}, "Transitional")
Map.addLayer(competitive_fc, {'color': '#c0dad7', 'opacity': 0.6}, "Competitive")

# Optional: zoom to the region automatically
region_all = distressed_fc.merge(at_risk_fc).merge(transitional_fc).merge(competitive_fc)
Map.centerObject(region_all, 8)

# ---- Add Legend ---- #
legend_dict = {
    "Distressed": "#ca727f",
    "At-Risk": "#eac69e",
    "Transitional": "#e9ebd4",
    "Competitive": "#c0dad7"
}

Map.add_legend(title="County Categorization", legend_dict=legend_dict)


# -------------------------------
# 3. Census Tracts
# -------------------------------
# tracts =ee.FeatureCollection("TIGER/2020/TRACT")
# Map.addLayer(tracts, {'color': 'white'}, "Census Tracts")


tracts = ee.FeatureCollection("TIGER/2020/TRACT")
states = ee.FeatureCollection("TIGER/2018/States")
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Function: Convert census tract codes like "503.02" → "050302"
def to_tract_code(code):
    code = str(code)
    if "." in code:
        a, b = code.split(".")
        return a.zfill(4) + b.zfill(2)
    else:
        return code.zfill(6)

# Define tract selections: (STATE, COUNTY, [TRACT LIST])
tract_selection = [
    ("Tennessee", "Marion", ["503.02", "502.01"]),
    ("Tennessee", "Sequatchie", ["601.04", "601.03"]),
    ("Tennessee", "Warren", ["9307"]),
    ("Tennessee", "Coffee", ["9701", "9709"]),
    ("Tennessee", "DeKalb", ["9202.02", "9202.01", "9203"]),

    ("Alabama", "Marshall", ["310.02", "311", "308.04", "308.03"]),
    ("Alabama", "Jackson", ["9507", "9508", "9506.02", "9503.02", "9502", "9501.01"])
]

# Build feature collection of selected tracts
selected_tracts = ee.FeatureCollection([])

for state_name, county_name, tract_list in tract_selection:

    # Get state FP code
    state_fp = ee.Feature(
        states.filter(ee.Filter.eq("NAME", state_name)).first()
    ).get("STATEFP")

    # Get county FP code
    county_fp = ee.Feature(
        counties.filter(
            ee.Filter.And(
                ee.Filter.eq("NAME", county_name),
                ee.Filter.eq("STATEFP", state_fp)
            )
        ).first()
    ).get("COUNTYFP")

    # Convert codes
    tract_codes = [to_tract_code(t) for t in tract_list]

    # Filter tracts
    fc = tracts.filter(
        ee.Filter.And(
            ee.Filter.eq("STATEFP", state_fp),
            ee.Filter.eq("COUNTYFP", county_fp),
            ee.Filter.inList("TRACTCE", tract_codes)
        )
    )

    selected_tracts = selected_tracts.merge(fc)

# Style visualization
tract_style = selected_tracts.style(
    color='000000',
    fillColor='f8dd6f80',   # 80 hex = ~50% transparent
    width=1
)

Map.addLayer(tract_style, {}, "Distressed Census Tracts (Selected)")

# Add Legend entry
legend_dict_update = {
    "Distressed Census Tract Areas FY 26": "#f8dd6f"
}

Map.add_legend(title="County & Tract Categorization", legend_dict=legend_dict_update)



Map


Map(center=[35.13408219642361, -85.9441932349594], controls=(WidgetControl(options=['position', 'transparent_b…

Pub map

In [9]:
!pip install geemap rasterio geopandas contextily matplotlib pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 73.4 MB/s eta 0:00:00


In [13]:
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import box, Point
from PIL import Image
import os

In [14]:
# Your bounding box (polygon)
bbox_coords = [
    [-86.8826120358268, 34.12771877579682],
    [-84.2953317623893, 34.12771877579682],
    [-84.2953317623893, 36.18887300565079],
    [-86.8826120358268, 36.18887300565079],
    [-86.8826120358268, 34.12771877579682]
]
bbox = ee.Geometry.Polygon([bbox_coords])

# Also create a simple shapely/GeoDataFrame version for matplotlib extents and CRS conversions
minx = -86.8826120358268; miny = 34.12771877579682
maxx = -84.2953317623893; maxy = 36.18887300565079
bbox_shp = box(minx, miny, maxx, maxy)
bbox_gdf = gpd.GeoDataFrame({"geometry":[bbox_shp]}, crs="EPSG:4326")

# Output folder for intermediate exports
out_folder = "/content/map_exports"
os.makedirs(out_folder, exist_ok=True)


In [18]:
export_locaiton = "/content/drive/MyDrive/ARSIE_MAP_Creation"

In [19]:
# --- GHSL URBAN ---
ghsl_img = (
    ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_C")
    .filter(ee.Filter.eq("year", 2018))
    .select("built_characteristics")
    .first()
)

ghsl_urban = ghsl_img.updateMask(ghsl_img.gte(4)).clip(bbox)

# --- Forest ---
forest = (
    ee.Image("UMD/hansen/global_forest_change_2022_v1_10")
    .select("treecover2000")
    .clip(bbox)
)

# --- EPA ECOREGIONS (use ecoregion ID, NOT shape_area) ---
eco = ee.FeatureCollection("EPA/Ecoregions/2013/L4")
eco_image = ee.Image().paint(eco, "US_L4CODE").clip(bbox)

# --- EXPORT TO GOOGLE DRIVE (correct method for large rasters) ---

geemap.ee_export_image_to_drive(
    ghsl_urban,
    description="ghsl_urban_export",
    folder=export_locaiton,
    fileNamePrefix="ghsl_urban",
    scale=10,
    region=bbox
)

geemap.ee_export_image_to_drive(
    forest,
    description="forest_treecover_export",
    folder=export_locaiton,
    fileNamePrefix="forest_2000",
    scale=30,
    region=bbox
)

geemap.ee_export_image_to_drive(
    eco_image,
    description="eco_regions_export",
    folder=export_locaiton,
    fileNamePrefix="eco_regions",
    scale=30,
    region=bbox
)

print("✅ Export tasks submitted. Open Earth Engine task panel or check Drive > export_locaiton.")


✅ Export tasks submitted. Open Earth Engine task panel or check Drive > GEE_Exports.


In [21]:
# import ee
# import geemap
# import os
# import geopandas as gpd

# ee.Initialize()

# -------------------------------
# Parameters
# -------------------------------
bbox = [
    [-86.8826120358268, 34.12771877579682],
    [-84.2953317623893, 34.12771877579682],
    [-84.2953317623893, 36.18887300565079],
    [-86.8826120358268, 36.18887300565079],
    [-86.8826120358268, 34.12771877579682]
]

export_location = "/content/drive/MyDrive/ARSIE_MAP_Creation"
os.makedirs(export_location, exist_ok=True)

# -------------------------------
# Load TIGER collections
# -------------------------------
counties_fc = ee.FeatureCollection("TIGER/2018/Counties")
states_fc = ee.FeatureCollection("TIGER/2018/States")
tracts_fc = ee.FeatureCollection("TIGER/2020/TRACT")

# -------------------------------
# Helper functions
# -------------------------------
def to_tract_code(code):
    """Convert census tract code like '503.02' → '050302'."""
    code = str(code)
    if "." in code:
        a,b = code.split(".")
        return a.zfill(4) + b.zfill(2)
    else:
        return code.zfill(6)

# Convert bbox to ee.Geometry
bbox_geom = ee.Geometry.Polygon(bbox)

# -------------------------------
# Selected census tracts (full list of 20)
# -------------------------------
tract_selection = [
    ("Tennessee","Marion",["503.02","502.01"]),
    ("Tennessee","Sequatchie",["601.04","601.03"]),
    ("Tennessee","Warren",["9307"]),
    ("Tennessee","Coffee",["9701","9709"]),
    ("Tennessee","DeKalb",["9202.02","9202.01","9203"]),
    ("Alabama","Marshall",["310.02","311","308.04","308.03"]),
    ("Alabama","Jackson",["9507","9508","9506.02","9503.02","9502","9501.01"])
]

# -------------------------------
# Build FeatureCollections
# -------------------------------
selected_tracts_fc = ee.FeatureCollection([])
selected_counties_fc = ee.FeatureCollection([])

for state_name, county_name, tract_list in tract_selection:

    # State FP
    state_fp = ee.Feature(states_fc.filter(ee.Filter.eq("NAME", state_name)).first()).get("STATEFP")

    # County FP
    county_fc = counties_fc.filter(
        ee.Filter.And(
            ee.Filter.eq("NAME", county_name),
            ee.Filter.eq("STATEFP", state_fp)
        )
    )
    county_fp = ee.Feature(county_fc.first()).get("COUNTYFP")

    # Merge county into selected counties
    selected_counties_fc = selected_counties_fc.merge(county_fc)

    # Format tract codes
    tract_codes = [to_tract_code(t) for t in tract_list]

    # Filter tracts
    tracts_fc_filtered = tracts_fc.filter(
        ee.Filter.And(
            ee.Filter.eq("STATEFP", state_fp),
            ee.Filter.eq("COUNTYFP", county_fp),
            ee.Filter.inList("TRACTCE", tract_codes)
        )
    )
    selected_tracts_fc = selected_tracts_fc.merge(tracts_fc_filtered)

# -------------------------------
# Filter by bounding box
# -------------------------------
selected_counties_fc = selected_counties_fc.filterBounds(bbox_geom)
selected_tracts_fc = selected_tracts_fc.filterBounds(bbox_geom)

# -------------------------------
# State outlines (no fill)
# -------------------------------
alabama_f = ee.Feature(states_fc.filter(ee.Filter.eq('NAME', 'Alabama')).geometry().intersection(bbox_geom))
tennessee_f = ee.Feature(states_fc.filter(ee.Filter.eq('NAME', 'Tennessee')).geometry().intersection(bbox_geom))

# -------------------------------
# Population centers + institutions
# -------------------------------
points = [
    ("The University of Alabama in Huntsville", -86.64508109480721,34.724106096441574),
    ("Land Trust of North Alabama", -86.59202340173793,34.70635398758348),
    ("University of the South Sewanee", -85.92031912702664,35.20364504855291),
    ("Appalachian Conservation Institute", -85.66909384042783,35.24017428765534),
    ("Huntsville",-86.5861,34.7304),
    ("Scottsboro",-86.0008,34.6445),
    ("Stevenson",-85.8039,34.8298),
    ("Albertville",-86.2567,34.2671),
    ("South Pittsburg",-85.6792,35.0140),
    ("Sewanee",-85.9211,35.1795),
    ("Tracy City",-86.0105,35.3241),
    ("Monteagle",-85.8161,35.2307),
    ("Jasper",-86.4727,34.4696),
    ("Sequatchie",-85.4799,35.4501)
]

pt_gdf = gpd.GeoDataFrame(
    {'name':[p[0] for p in points],
     'lon':[p[1] for p in points],
     'lat':[p[2] for p in points]},
    geometry=gpd.points_from_xy([p[1] for p in points],[p[2] for p in points]),
    crs="EPSG:4326"
)

# -------------------------------
# Export FeatureCollections to GeoJSON
# -------------------------------
geemap.ee_export_vector(selected_counties_fc, filename=os.path.join(export_location, "counties_selected.geojson"))
geemap.ee_export_vector(selected_tracts_fc, filename=os.path.join(export_location, "selected_tracts.geojson"))

# Export state outlines as GeoJSON (small, safe via getInfo)
import json
with open(os.path.join(export_location, "alabama_outline.geojson"), "w") as f:
    json.dump(ee.FeatureCollection([alabama_f]).getInfo(), f)

with open(os.path.join(export_location, "tennessee_outline.geojson"), "w") as f:
    json.dump(ee.FeatureCollection([tennessee_f]).getInfo(), f)

# Save points
pt_gdf.to_file(os.path.join(export_location, "city_points.geojson"), driver="GeoJSON")

print("✅ Vector exports complete. All 20 census tracts included.")


Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/ARSIE_MAP_Creation/counties_selected.geojson
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/ARSIE_MAP_Creation/selected_tracts.geojson
✅ Vector exports complete. All 20 census tracts included.


In [22]:
import os
from rasterio.plot import show
import geopandas as gpd
import rasterio
from rasterio.plot import show
from matplotlib.colors import ListedColormap, BoundaryNorm
import numpy as np

# Paths
ghsl_path = os.path.join(export_location, "ghsl_urban.tif")
forest_path = os.path.join(export_location, "forest_treecover2000.tif")
eco_path = os.path.join(export_location, "eco_painted.tif")
counties_path = os.path.join(export_location, "counties_selected.geojson")
tracts_path = os.path.join(export_location, "selected_tracts.geojson")
al_path = os.path.join(export_location, "alabama_outline.geojson")
tn_path = os.path.join(export_location, "tennessee_outline.geojson")
points_path = os.path.join(export_location, "city_points.geojson")

# Read rasters
ghsl_src = rasterio.open(ghsl_path)
forest_src = rasterio.open(forest_path)
eco_src = rasterio.open(eco_path)

# Read vectors
counties_gdf = gpd.read_file(counties_path).to_crs(epsg=3857)
tracts_gdf = gpd.read_file(tracts_path).to_crs(epsg=3857)
al_gdf = gpd.read_file(al_path).to_crs(epsg=3857)
tn_gdf = gpd.read_file(tn_path).to_crs(epsg=3857)
points_gdf = gpd.read_file(points_path).to_crs(epsg=3857)
bbox3857 = bbox_gdf.to_crs(epsg=3857).geometry.total_bounds  # used for axes limits

print("Loaded files. Extent (EPSG:3857):", bbox3857)


RasterioIOError: /content/drive/MyDrive/ARSIE_MAP_Creation/ghsl_urban.tif: No such file or directory

In [ ]:
# Layout parameters
fig_w = 12   # figure width inches (adjust)
fig_h = 9    # figure height inches (adjust)
dpi = 600    # publication DPI (set to 300-600 as you like)
fig = plt.figure(figsize=(fig_w, fig_h), dpi=150)  # dpi here only for screen; savefig uses `dpi` arg below

# GridSpec: main map left large, three small axes stacked on right
import matplotlib.gridspec as gridspec
gs = gridspec.GridSpec(nrows=3, ncols=4, width_ratios=[3,3,3,1.2], height_ratios=[1,1,1], hspace=0.18, wspace=0.08)

# Main map uses columns 0-2 and rows 0-2 (span)
ax_main = fig.add_subplot(gs[:, :3])
ax_main.set_title("Primary Study Area — Counties & Selected Tracts", fontsize=12, weight='bold')

# Plot OSM basemap via contextily on ax_main
# We'll set main extent from bbox3857
minx, miny, maxx, maxy = bbox3857
ax_main.set_xlim(minx, maxx)
ax_main.set_ylim(miny, maxy)

# Add OSM background
ctx.add_basemap(ax_main, source=ctx.providers.OpenStreetMap.Mapnik, zoom=9)

# Plot counties (categorical fills)
cat_colors = {
    "Distressed":"#ca727f",
    "At-Risk":"#eac69e",
    "Transitional":"#e9ebd4",
    "Competitive":"#c0dad7"
}
for cat, color in cat_colors.items():
    sub = counties_gdf[counties_gdf['cat']==cat]
    if len(sub):
        sub.plot(ax=ax_main, facecolor=color, edgecolor='k', linewidth=0.4, alpha=0.6)

# Plot selected tracts on top
tracts_gdf.plot(ax=ax_main, facecolor="#f8dd6f", edgecolor='0.15', linewidth=0.6, alpha=0.5)

# State outlines
al_gdf.boundary.plot(ax=ax_main, color='black', linewidth=1.2)
tn_gdf.boundary.plot(ax=ax_main, color='black', linewidth=1.2)

# Add legend for counties and tracts (manual handles)
import matplotlib.patches as mpatches
handles = [mpatches.Patch(color=v, label=k, alpha=0.7) for k,v in cat_colors.items()]
handles.append(mpatches.Patch(color="#f8dd6f", label="Distressed Census Tracts FY26", alpha=0.6))
ax_main.legend(handles=handles, loc='lower left', frameon=True, fontsize=8)

# ADD scale bar (simple)
scalebar_x = minx + (maxx-minx)*0.03
scalebar_y = miny + (maxy-miny)*0.03
# 10 km scale bar in meters (approx)
ax_main.plot([scalebar_x, scalebar_x + 10000], [scalebar_y, scalebar_y], 'k-', linewidth=3)
ax_main.text(scalebar_x, scalebar_y - (maxy-miny)*0.02, "10 km", fontsize=8)

# === INSET 1: Forest cover ===
ax1 = fig.add_subplot(gs[0, 3])
ax1.set_title("Inset 1: Forest cover", fontsize=9)
# plot forest raster with a green palette
forest_img = forest_src.read(1)
# Mask no-data
forest_img = np.where(forest_img<0, np.nan, forest_img)
# show with imshow using extent
extent = (minx, maxx, miny, maxy)
im1 = ax1.imshow(forest_img, extent=extent, origin='upper', cmap='Greens', vmin=0, vmax=100)
ax1.set_xticks([]); ax1.set_yticks([])
# Colorbar small
cb1 = fig.colorbar(im1, ax=ax1, fraction=0.045, pad=0.01)
cb1.set_label("Tree cover %", fontsize=7)

# === INSET 2: EPA ecoregions (categorical) ===
ax2 = fig.add_subplot(gs[1, 3])
ax2.set_title("Inset 2: EPA ecoregions (L4)", fontsize=9)
eco_img = eco_src.read(1)
# create categorical colormap: pick N unique values in eco_img within bounding box
unique_vals = np.unique(eco_img[~np.isnan(eco_img)])
# build colormap automatically
cmap_vals = plt.cm.get_cmap('tab20', len(unique_vals))
cmap = ListedColormap(cmap_vals.colors)
# map unique values to 0..n-1
val_to_idx = {v:i for i,v in enumerate(unique_vals)}
eco_plot = np.full_like(eco_img, np.nan)
for v, i in val_to_idx.items():
    eco_plot[eco_img==v] = i
im2 = ax2.imshow(eco_plot, extent=extent, origin='upper', cmap=cmap)
ax2.set_xticks([]); ax2.set_yticks([])
# Build legend for ecoregions present
ec_handles = []
for v,i in val_to_idx.items():
    ec_handles.append(mpatches.Patch(color=cmap(i), label=f"eco {int(v)}"))
ax2.legend(handles=ec_handles, fontsize=6, loc='lower left', frameon=True)

# === INSET 3: Population centers + Urban layer + logos ===
ax3 = fig.add_subplot(gs[2, 3])
ax3.set_title("Inset 3: Urban centers & population", fontsize=9)
ghsl_img = ghsl_src.read(1)
# Mask zeros/nodata
ghsl_img = np.where(ghsl_img==0, np.nan, ghsl_img)
im3 = ax3.imshow(ghsl_img, extent=extent, origin='upper', cmap='hot')  # you may want a custom palette
# plot points
points_gdf.plot(ax=ax3, markersize=30, color='blue', alpha=0.8)
# label key cities (use smaller font for clarity)
for idx, row in points_gdf.iterrows():
    ax3.text(row.geometry.x + 500, row.geometry.y + 500, row['name'], fontsize=6)

ax3.set_xticks([]); ax3.set_yticks([])

# Place logos on inset 3 (reads from your Drive path). Adjust positions as needed.
logo_folder = "/content/drive/MyDrive/Writing/Grants/grants/Appalachian Regional Commission (ARC) Appalachian Regional Initiative for Stronger Economies (ARISE)/logo_folder"
# Example filenames - adjust to actual filenames in that folder
logo_files = [f for f in os.listdir(logo_folder) if f.lower().endswith(('.png','.jpg','.jpeg'))]
# place up to 3 logos in lower right corner of inset 3
logo_plot_x = maxx - (maxx-minx)*0.08
logo_plot_y = miny + (maxy-miny)*0.08
for i, lf in enumerate(logo_files[:3]):
    img = Image.open(os.path.join(logo_folder, lf))
    img.thumbnail((80,80), Image.ANTIALIAS)
    # draw image on axis using inset axes
    ax3.imshow(img, extent=(logo_plot_x + i* (maxx-minx)*0.02,
                            logo_plot_x + i*(maxx-minx)*0.02 + (maxx-minx)*0.02,
                            logo_plot_y,
                            logo_plot_y + (maxy-miny)*0.03), zorder=10)

# final touches
for ax in [ax_main, ax1, ax2, ax3]:
    ax.set_axis_off()

# Save final PNG
output_png = "/content/Area_Development_Figure.png"
plt.savefig(output_png, dpi=dpi, bbox_inches='tight')
plt.close(fig)
print("Saved publication figure to:", output_png)
